## Summary

`TraceManager` collects all the called functions with useful information. <br/>
The information includes the inputs to the called functions.


# Body


You can check the code of `script` in the link:

[script.py](./script.py)

In [1]:
from script import lvl1
from inspect import getsource
from crimson.tracer.filter import FrameFilter
from crimson.tracer import TraceManager, TraceEvent


## Trace

### Without `frame_filter`


In [2]:
TraceManager.run_trace(lvl1)
trace_result = TraceManager.get_trace("lvl1")

trace_result

[TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=8, event='call', arg=None, level=0, locals={}, custom=None),
 TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=13, event='line', arg=None, level=0, locals={}, custom=None),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=74, event='call', arg=None, level=1, locals={'self': Node('/root'), 'name': 'root', 'parent': None, 'children': None, 'kwargs': {}}, custom=None),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=75, event='line', arg=None, level=1, locals={}, custom=None),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=76, event='line', arg=None, level=1, locals={

### With `frame_filter`


In [3]:
frame_filter = FrameFilter(
    exclude=["python3", "site-packages"]
)

print(getsource(FrameFilter))
TraceManager.set_frame_filter(frame_filter)

TraceManager.run_trace(lvl1)
trace_result = TraceManager.get_trace("lvl1")

trace_result


class FrameFilter:
    """
    A class for filtering stack frames during tracing.

    - If `include` is `None`, **all files are included**, and only `exclude` is applied.
    - If `include` is set, only matching patterns are included, and then `exclude` is applied.

    Args:
        include (Optional[List[str]]): A list of filename patterns to include (e.g., `["src/", "lib/"]`).
            If `None`, all files are included.
        exclude (Optional[List[str]]): A list of filename patterns to exclude (e.g., `["test/", "logs/"]`).
            If `None`, nothing is excluded.

    Examples:
        # 1. Include all files except specific patterns
        filter_all = FrameFilter(exclude=["test/", "logs/"])

        # 2. Include only specific directories and exclude certain files
        filter_specific = FrameFilter(include=["src/", "lib/"], exclude=["logs/"])

        # 3. Exclude specific file patterns
        filter_exclude_only = FrameFilter(exclude=["venv/", "test_"])
    """

    

[TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=8, event='call', arg=None, level=0, locals={}, custom=None),
 TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=13, event='line', arg=None, level=0, locals={}, custom=None),
 TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=15, event='line', arg=None, level=0, locals={}, custom=None),
 TraceEvent(function='lvl2_1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=25, event='call', arg=None, level=1, locals={}, custom=None),
 TraceEvent(function='lvl2_1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=31, event='line', arg=None, level=1, locals={}, custom=None),
 TraceEvent(function='lvl2_1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=31, event='return', arg=1, level=1, locals={}, custom=None),
 Trace

### With `name`

if `name` is `None`, the name of the func is used instead

In [4]:
TraceManager.run_trace(func = lvl1, name = 'custom name')
TraceManager.traces.keys()

dict_keys(['lvl1', 'custom name'])

### Edit Trace Event

All the available arguments in the tracked functions are stored as `locals`. <br/>
It can cause the `out of memory`. <br/>
To deal with this issue, set `event_editor`.

To annotate the `TraceEvent.custom` field, set `custom_type`.


In [5]:
def event_editor(trace_event: TraceEvent) -> TraceEvent:
    trace_event.custom = len(trace_event.locals)
    trace_event.locals = {}

    return trace_event

def frame_filter_dummy(frame):
    return True

TraceManager.set_event_editor(event_editor=event_editor)
TraceManager.set_frame_filter(frame_filter_dummy)

TraceManager.run_trace(func = lvl1, name = 'lighter locals')
trace_result = TraceManager.get_trace('lighter locals', custom_type=int)
trace_result

[TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=8, event='call', arg=None, level=0, locals={}, custom=0),
 TraceEvent(function='lvl1', filename='/home/crimson/manager/crimson/tracer/example/script.py', lineno=13, event='line', arg=None, level=0, locals={}, custom=0),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=74, event='call', arg=None, level=1, locals={}, custom=5),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=75, event='line', arg=None, level=1, locals={}, custom=0),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib/python3.12/site-packages/anytree/node/node.py', lineno=76, event='line', arg=None, level=1, locals={}, custom=0),
 TraceEvent(function='__init__', filename='/home/crimson/miniconda3/envs/tracer/lib

IDE knows the type of `TraceEvent.custom`.

<!--StartFragment--><div style="color: #cccccc;background-color: #1f1f1f;font-family: Consolas, 'Courier New', monospace;font-weight: normal;font-size: 14px;line-height: 19px;white-space: pre;"><div><span style="color: #9cdcfe;">trace_result</span><span style="color: #cccccc;">[</span><span style="color: #b5cea8;">0</span><span style="color: #cccccc;">].</span><span style="color: #9cdcfe;">custom</span><span style="color: #cccccc;">.</span><span style="color: #9cdcfe;">imag</span></div></div><!--EndFragment-->
